## HPC - Challenge 1
HS22 - Data Scince

## Part 1 - Kafka Cluster and Application Setup

#### Introduction
The first client (data_producer1) has company cars and wants to track their location with a gps tracker. Additionally, he wants to track the cars speed, acceleration and breaking. The data is sent below one hertz (hz) to Kafka. The speed of received data is variable but slower than one hertz. (Many cars produce data, but one manager receives and collects data)

The second client has IOT-data from a heating unit, sending its data every minute to Kafka (for testing the speed to send data is increased). The data is regularly read from different departments. The planning department (A) reads data twice a day, while the maintenance department (B) reads the data once an hour. (One unit produces data and different department read it)


#### Exercise 4.
Run the Kafka tutorial of the notebook. Use Kafdrop to monitor/explore your cluster, topics, and messages. For example, start and stop individual brokers (via Docker Desktop) or change Kafka parameters such as the replication factor and watch how the cluster behaves via Kafdrop. Describe your experiments and observations in about 150 words.

Answer:
Experiment 1:
All brokers are running, kafka_tutorial.ipynb is executed and producing default messages on topic “names”.
Observations:
At the beginning broker1 (the defined broker) handles 92% of the leader partitions. After a few messages all the three brokers handling around 31-38% of the leading partitions. It seems the topic is randomly but evenly distributed to the partitions from the three available brokers. Those assignments don’t change with more massages.

Experiment 2:
While producing message for topic “names” first broker2 is stopped and then secondly broker3 as well.
Observations:
1.) Kafkadrop1 UI shows the missing broker2. Messages still being sent successfully. Broker1 and Broker3 are left and handle each 46% of leading partitions (total 92% where are the other 8%?). If the stopped broker2 runs again assingments to partitions happen not immediately, some latency or not necessary anymore in this small experiment. 2.) Kafkadrop1 UI shows broker2 and 3 missing, messages are still received from Kafka and broker1 handling all partitions (92%). With Acknowledgment default set to 1 it’s sending messages to Kafka much slower now. So, if brokers die the other brokers can handle the topics but at a slower rate, scalability with more brokers (parallel processing?)


#### Exercise 5.
Write two different data generator functions, which regularly send messages containing data. One generator should send messages at least every second (1Hz). Choose yourself which data is sent. The application of the data can be chosen freely, but choose a good mixture of a simple and a complex message. The data should be variable. The data generator can send simulated data or real data. Use suitable Kafka components and meaningful names of functions, variables etc. for the implementation.

### Data generator 1
To keep it simple instead of gps-data I use a sinus function as single data point. Below the csv-file is generated:

![](part1/Sinus_daten.jpg)

### Data generator 2

The second data generator producer takes data from a heating measurements file. It's a time series data stream with time stamps and many other technical measurements.

![](part1/Heizungdaten.jpg)

#### Exercise 6.
The Jupiter Notebooks 'data_producer_1', 'data_producer_2', 'data_consumer1', 'data_consumer2' sending and receiving data to and from Kafka, received data is stored in a cdv-file. Run those four notebooks and check the print messages.

#### Exercise 7.

![](overview_all_scaled.jpg)

**What are the tasks of the components?**
Producer: generates the data and sends it with a topic name to a broker in Kafka.
Kafka: is a messaging system and distributes messages along its platform. Kafka runs as a cluster of one or more servers that can span multiple datacenters or cloud regions.
Broker: Kafka has storage Layers called broker. Brokers distribute the incoming messages. For bigger applications more brokers can be used to increase scalability and reliability.
Topics: Topics are named containers for messages, new messages are appended to the topic. For example, a topic can be seen as a folder in a filesystem, and the messages are the files in that folder. A topic is replicated to multiple brokers.
Partitions: A topic can be partitioned into smaller junks. Partitions are the fundamental mechanism, they enable parallelization of topics and enabling high message throughput.
Consumer: can read messages from one or more topics. Consumers are independent from the producer a can decide at which speed data is taken

**Funktionsweise?**
Öffne den Link der auf den Jupyter Container führt. Im Fileverzeichniss des Containers stehen verschiedene Notebooks zur Verfügung. (Im Ordnerverzeichniss unter: part1/notebooks zu finden).

Die Notebooks "data_producer_1a" und "data_producer_2" lesen von den csv-files und übergeben die Daten an Kafka (Push Message). Führe die Notebooks aus und prüfe, ob die Daten erfolgreich verschickt werden ("Message published successfully to topic: data_gen1").
In den data_consumer Notebooks werden die Daten wieder aus Kafka gelesen (Pull Message). Die Empfänger/Consumer können in unterschiedlichem Zeitabständen die Daten von Kafka lesen. Für den Kunden mit den Heizungsdaten werden alle 2 Sekunden Daten verschickt, consumer2a holt die Daten alle 4 Sekunden während consumer2b die Daten nur alle 10 Sekunden benötigt.
Die Consumer werden beenden, wenn die Producer keine neuen Daten mehr versenden, bei neuen Daten müssen daher auch die Consumer neu gestartet werden. Dies wurde eingebaut, um die Schleifenprozesse einfacher zu beenden. In der Praxis würde der wohl weiter laufen und auf Daten warten.

Bugs:
Wenn derzeit die Consumer Notebooks neu gestartet werden, werden alle Daten, die in der Kafka Queue noch vorhanden sind, erneut eingelesen und gespeichert. Bei einem Neustart wird ein neuer Cunsumer erstellt womit, alle Messages von Kafka gelesen werden. Es wurde versucht mit einer "client_id" einen Consumer "anzumelden" um die Daten seit der letzten Anfrage zu erhalten. Funktioniert derzeit noch nicht.

In Plannung:
- derzeit keine Partitionen für Topics vergeben, könnte für topic 2 Sinn machen.
- die empfangenen Daten werden in einem File gespeichert und gehen verloren, wenn der Container neu gestartet wird. Ein Script welches die Daten vom Container herunterlädt wäre nützlich. (Port Container, File kopieren)


#### Bonus
Use other serializers/deserializers instead of JSON for the messages.

## Part  2 - Communication Patterns
In der Aufgabenstellung soll eine Alternative zu Kafka implementiert werden. Da für mich Kafka neu ist und von Part 1 noch ein paar Bugs und nützliche Erweiterungen offen sind, würde ich hier gerne die Kenntnisse zu Kafka vertiefen und weiter Optionen ausprobieren. Somit wird für Part2 weiter Kafka verwenden, RabbitMQ wird eingelesen aber vorläufig nicht implementiert.

#### Exercise 2.
Pack your rewritten application into containers. Describe the used containers and how they interact with each other using a diagram.

![](overview_2_scaled.jpg)

Die einzelnen Docker Container sollen die unabhängigen Prozesse darstellen. Verschiedene Fahrzeuge (part2_producer1a) und Heizungen (part_producer2) generieren Daten und senden dies über Netzwerke an Kafka. Diese Prozesse laufen unabhängig und kontinuierlich. Die Verarbeitungs- und/oder Leseprozesse werden in separate Container aufgebaut. Im Planning Departement (part_consumer2a) werden die Daten gelesen und in einem File gespeichert, während das Maintenance Departement die Daten lediglich liest. Die Consumer können, zum Beispiel für Wartungszwecke, ohne Probleme gestoppt werden. Bei einem Neutstart werden die letzten noch nicht erhaltenen Daten von Kafka gelesen (Daten bis 2 Wochen in Kafka zur Verfügung).

Alle Container haben eine gemeinsame Schnittstelle, Kafka. Die Consumer Seite ist indirekt von den Producern abhängig, insofern dass die Producer Daten generieren sollten, aber es bestehen keine Abhängigkeiten innerhalb des Containers und deren Umgebung (Dependencies and stuff).
Folgende Startabhängigkeiten wurden für das docker-compose file festgelegt:
1. starten der Broker
1. starten der Producer
2. starten der Consumer

Da ein compose file verwendet wurde, wurden alle Container praktischerweise im selben Netzwerk angelegt. Für Container ausserhalb müsste ansonsten ein Netzwerk angelegt werden.



**Ausführen Software Part 2**
Führe "docker-compose up" aus. Die Container werden entsprechend der Übersichtsgrafik aufgebaut und gestartet. Um die Funktionsfähigkeit zu prüfen, können die einzelnen Container im Docker Windows geöffnet werden. In den jeweiligen Docker  zeigen verschiedene Messages was aktuell stattfindet.

**Verbesserungen / Anpassungen**

Bugs:
- (behoben) Die consumer files starten nicht automatisch, werden derzeit von Hand gestartet (docker windows -> CLI -> "python data_consumer1.py"). container console und python print werden nicht in echtzeit verarbeitet. print(, flush=True) behebt dies.
- (behoben) while loop der Consumer files hängt. Consumer1: while loop und "consumer_timeout_ms" entfernt, Consumer2a/2b: Um das gewünschte Zeitintervall mit dem while loop zu erstellen sollte die "consumer_timeout_ms" Zeit kleiner als die der Producer sein.

In Plannung:
- derzeit keine Partitionen für Topics vergeben, könnte für topic 2 Sinn machen.
- die empfangenen Daten werden in einem File gespeichert und gehen verloren, wenn der Container neu gestartet wird. Ein Script welches die Daten vom Container herunterlädt wäre nützlich. (Port Container definiert)
- Welche Vorteile bringen Subscriber Gruppen?

#### Exercise 3.

**Die Vorteile**
von Publish-Subscriber Kommunikation Systemen ist das Reduzieren von Abhängigkeiten oder die Entkopplung von Applikationsteilen. Die Publisher-seite kann sich auf das Veröffentlichen von Daten konzentrieren (Message -> Topic). Je nach Einstellungen können Rückmeldungen der Subscriber verlangt werden um zu prüfen, ob die Daten angekommen sind. Die Subscriber schreiben sich für die Topics ein, die für sie relevant sind. Es können beliebig Subscriber dazukommen oder fernbleiben.

**Die Nachteile**
sind wohl das Prüfen, ob die Nachrichten komplett und korrekt beim Subscriber angekommen ist. Garantieren kann der Publisher dies durch die Entkopplung der Systeme nicht mehr. Es gibt verschiedene Einstellungen die Antworten von Subscriber fordern und prüfen können. Muss aber auf eine Antwort des Subscribers gewartet werden kostet dies Zeit. Auch sind sensible Daten, wie kritische Probleme oder Fehler, stärker von diesem Nachteil betroffen und Daten könnten evtl. nicht zeitnah bei den Teilnehmern ankommen. Weiter ist der Ausfall von Publisher, wenn nicht regelmässig Daten versendet werden, ein Problem. Für die Teilnehmer ist es sehr schwierig einen solchen Ausfall zu bemerken.

**Die Skalierbarkeit**
in Kafka kann mit zusätzlichen Brokern umgesetzt werden (horizontales skalieren, meistens verwendet). Die Performance der Broker kann auch durch Erhöhen der Partitionen von Topics verbessert werden (vertikales skalieren). Ein Publish-Subscriber System skaliert wesentlich besser als ein Verteilsystem, bei dem die Empfänger einzeln behandelt werden (Mail/SMS an 10000 Kunden senden).

#### Bonus
Show how your container setup could be integrated into a container orchestration system (such as Kubernetes) and how it would profit from this. Or show how you could replace some components with cloud-based offers and what changes/considerations come with this.

## Part 3 - Performance Analysis and Evaluation
1. Change the scalability of your communication framework by changing the used resources. (E.g.: add brokers and/or zookeepers and play with parameters in the docker-compose file. Use orders of magnited more producers/consumers.) Perform 2-3 experiments with different configurations. Document your experiments and their results appropriately (e.g. by video recording). Discuss the results in about 200 words.

**Experimente**
In den folgenden Versuchen soll getestet werden, welchen Nachrichtenfluss Kafka mit unterschiedlicher Anzahl Broker aufrechterhalten kann. Die Anzahl Nachrichten werden künstlich vervielfacht, der "Producer2" im Compose-File wird dafür mehrfach in seperaten Containern dupliziert und die Sendefrequenz auf 0.01 Sekunden heruntergesetzt.

Um den Nachrichtenfluss zu messen wurde ein Topic `Performance` aufgesetzt. Direkt nachdem der Producer2 seine Daten versendet hat, wird auf dem Topic `Performance` den Zeitpunkt des versendens und den hostnamen publiziert. Durch das Vervielfachen des Producer2 mit mehreren Containern, wird der Hostname benötigt, um zwischen den Producern unterscheiden zu können. Derzeit nicht benötigt da die gesamt Summe der Nachrichten geprüft werden, der Nachrichtenfluss je Producer könnte aber auch verwendet werden.

Ausführen der Versuche:
1. Das Compose-File wird entsprechend den Versuchen A-H angepasst.
2. Mit `docker-compose up --build` die Images neu bauen und die Container starten
3. Im Container  `jupyter_perf` das Notebook starten, dieses zeigt den Nachrichtenfluss an und aktualisiert sich alle fünf Sekunden.

**Folgend verschiedene Versuche:**
Versuche A. Fünf zusätzliche Producer (part3_producer3-7) wurden erstellt und drei Broker verwendet:
Resultat A: Auf dem Topic `Performance` kommen um die 400 Nachrichten / sek an. Soweit keinen Einfluss bemerkbar.

![](part3/Performance_test_A.png)

Versuche B. Weitere sechs Producer werden erstellt (part3_producer8-13) und drei Broker verwendet:
Resultat B: Auf dem Topic `Performance` fährt der Nachrichtenfluss hoch und wird bei ca 800 Nachrichten / sek stabil. Keinen Einfluss bemerkbar, die Anzahl Nachrichten pro Sekunde variieren stärker als bei Versuch A. Erhöhen der Producer um Faktor 2 lässt hier somit auch den Nachrichtenfluss um ca. den Faktor 2 steigen.

![](part3/Performance_test_B.png)

Versuche C. "Broker4" wird beigefügt und Container neu gestartet, totale Broker = 4:
Resultat C: Auf dem Topic `Performance` fährt der Nachrichtenfluss hoch und wird wieder bei ca 800 Nachrichten / sek stabil. Der tiefere Nachrichtenfluss beim Starten der Container kommt wohl daher, dass 12 von 13 Producer den `Broker2` als bevorzugten Broker hinterlegt haben. Da aber vier Broker zur Verfügung stehen, verteilt Kafka die Nachrichten auf alle Broker, was wohl eine gewisse Zeit dauert. Für einen sauberen start wäre es wohl von Vorteil die bevorzugten Broker gleichmässig zu verteilen.

![](part3/Performance_test_C.png)

Versuche D. "Broker3" wird heruntergefahren, totale Broker = 3 vorhanden:
Resultat D: Der Nachrichtenfluss nimmt wenig ab, von 800 auf ca. 750 Nachrichten / sec. Der Nachrichtenfluss variiert relativ stark zwischen 600 und 800 Nachrichten / sek.

![](part3/Performance_test_D.jpg)

Versuche E. "Broker2" wird heruntergefahren, totale Broker = 2 vorhanden:
Resultat E: Der Nachrichtenfluss fällt deutlich auf 500 Nachrichten / sek zusammen, fährt aber wieder hoch auf 800 mit derselben Spannweite wie in Test D

![](part3/Performance_test_E.jpg)

Versuche F. "Broker1" wird heruntergefahren, totale Broker = 1 vorhanden:
Resultat F: Keine Veränderungen für ca 10 Sekunden. Nachrichtenfluss bei 750 Nachrichten / sek. Vermutlich durch den `Replikationsfaktor=2` muss Kafka die Nachrichten nicht mehr verteilen. Nach 10 Sekunden bricht Broker 1 zusammen, Nachrichtenfluss fällt auf 0 mit einzelnen Nachrichten und behält diesen Status. Ein einzelner Broker kann die Applikation nicht handhaben.

![](part3/Performance_test_F.jpg)

Versuche G. Starten der Container mit fünf Brokern:
Resultat G: Nachrichtenfluss fährt hoch, stabil bei 800 Nachrichten / sek. Ähnlich zu Versuch A, jedoch stabiler als in Versuch B mit vier Broker und allen Producer.

![](part3/Performance_test_G.png)

Versuche H. "Broker4" wird heruntergefahren:
Resultat H: Nachrichtenfluss kurzzeitig auf 600 Nachrichten / sek. Sehr schnell wieder stabil bei 800 Nachrichten / sek. Ausfall von Broker4 kaum spürbar.

![](part3/Performance_test_H.jpg)

**Abschliessende Reflektion zu den Versuchen**
Aus den Versuchen wird ersichtlich, warum die Literatur mindestens drei Brokern empfiehlt. Durch den Replikationsfaktor, der >= 2 sein sollte, sind zwei Broker eigentliche schon das Minimum. Soll das System auch bei einem Broker Ausfall noch funktionieren, dann es schon mindestens drei Broker. Beim Aufbauen eines Nachrichtensystems macht es wohl Sinn, die Performance oder das Verhalten des Nachrichtenflusses zu testen anhand der Anzahl Broker zu messen. Der letzte Versuch H (einer von fünf Broker fällt aus) zeigt, dass wenn genügend Kapazität vorhanden ist (Kafka nicht am Limit arbeitet) ein Broker Ausfall sehr stabil und fast unbemerkt bearbeitet wird. Ein Indiz für ein System an der Grenze könnte evtl. Versuch D und E gezeigt haben. Der Nachrichtenfluss variiert sehr stark, ein ähnliches Verhalten wurde in Versuch F gesehen, wo der Nachrichtenfluss mit nur einem Broker komplett zusammenbrach.

Wort zu Performance Messung: Die Zeitstempel der Producer werden mit einem eigenen Topic an Kafka übermittelt. Somit verursacht die Messung selbst Aufwand für die Broker. Die Nachrichten sind klein, aber es schwierig einzuschätzen wie viel Performance die Messung kostet oder ob dieser Ansatz überhaupt skalierbar wäre. Interessant wäre noch eine Auswertung des Nachrichtenflusses der Consumer, vorstellbar wären Logfiles je Consumer und diese Zeitstempel zu vergleichen.



2. Analyze the performance of your application:

A. Data generators: measure the average time incl. standard deviation required by your data generator loop over several runs and loops. Determine a reasonable number of runs and loop executions for the performance analysis.
B. Data processor: determine which call of your processor takes the most time. Which 3 methods are called the most or needed the most time and how much time?
C. Data processor: create a profile of your processor code in a processor.prof file and create 1-2 visualizations of the profile (e.g. with SnakeViz).
D. Data generators, data processor, data sink: Create a profile of memory usage and output the first 3 most memory-hungry methods.

Für die Analyse des Data Producer2 wird ein Jupyter Notebooks aus Part 1 verwendet. Um die Produce und Consumer aus dem Notebook zu starte, wurde das  das compose-file angepasst, respt. die zusätzlichen Container mit Producer und Consumer auskommentiert.
1. starten des modifizierten compose-files (original Zustand)
2. jupyter nootebook "data_producer_2" öffnen und die Funktionen unter "Data Producer 2 Zeit Messungen" laufen lassen.

**A.**
Für die Messung wurde [timeit](https://linuxhint.com/timeit-jupyter-notebook/) verwendet. Die Verzögerung der Sendezeit von 1 Sekunde wurde entfernt. Da die Funktion des producer_xy nicht abbricht (Daten werden kontinuierlich gesendet) wurde ein Abbruch Kriterium bei i > 5 eingebaut. Timeit führt default 100-mal die Funktion aus und berechnet den Mittelwert und Standardabweichung der Laufzeit.
Die absoluten Messezeiten sind nicht sehr relevant, je nach System und dessen Auslastung werden andere Ergebnisse erzielt. Gleiche Funktionen mit unterschiedlicher Implementations können aber so verglichen werden. Für den Producer2 wurden folgende Messungen gemacht.

![](part3/timeit_producer2.jpg)


**C.**
[SnakeViz](https://jiffyclub.github.io/snakeviz/) Auswertung

Der grösste Teil der Laufzeit der Funktion `produce_xy` wird für das Senden der Nachrichten aufgewendet (`publish_message`). Das vermutete Bottleneck, das Lesen der Daten mit `pandas.read_csv()` bestätigt SnakeViz nicht. Da nur jeweils eine Zeile des Files gelesen und verschickt wird, taucht der Funktionsaufruf nicht mal in der Grafik auf.

Producer2
![](part3/SnakeViz_producer2.jpg)

Consumer2a
![](part3/SnakeViz_consumer2.jpg)

**D.**
Hierfür wurde das Package `memory_profiler` verwendet. Leider kann die Speicherauswertung nicht auf Funktionen innerhalb der Notebooks gemacht werden. Daher werden die Funktionen mit `%%file` in ein separates File geschrieben und ausgewertet.

![](part3/memory_profiler.jpg)



3. Did you detect bottlenecks? Describe and discuss 1-3 detected bottlenecks in about 200-300 words.

**Reflektion Part 3**
Beim Producer2 werden Zeilenweise Daten vom File gelesen und versendet. Innerhalb der Funktion `produce_xy` dauert das Senden von Nachrichten am längsten. Das Lesen der Daten hat keinen grossen Einfluss.
Die SnakeViz Analyse zum Consumer2a sind Namens technisch schwer zu interpretieren, es sieht aus als würde das Lesen der Nachrichten von Kafka die meiste Zeit benötigen (1.077 / 1.24 Sekunden). Schreiben der Daten als csv Datei belegt Platz 6 in Tabelle, benötigt aber gerade mal 0.0027 Sekunden.

Eine Analyse der Laufzeiten kann helfen, Zeitfresser zu identifizieren und Optimierungen nicht am falschen Ort anzusetzen. Für diese Anwendung würde es wohl Sinn machen, anstelle von einzelnen Zeilen, direkt grössere Datenpakete zu versenden. Der Durchsatz an Information könnte so erhöht werden.


"peak memory" wird mit ca. 110 MB gemessen, auch in memory_profiler werden 110 MB gesehen. Die einzelnen Methoden werden jeweils mit 0 MB gesehen. Das Datenfile umfasst ca. 100 MB Daten, die Vermutung liegt nahe, dass das File den Hauptteil dieser Messung ausmacht.

## Reflektion Mini-Challenge 1